In [110]:
# -*- coding: utf-8 -*-
# 1.1 Case Study
# For this Assignment you will be designing and developing a Database for a fantasy base building game. One example of such a game is Townsmen. For this Assignment you do not need to worry about implementing the game at all, we are just focusing on the database.
# The game is multiplayer with multiple players. Each player has a base, in which they can construct various buildings. These buildings produce resources which can be used to construct other buildings or can be sold for currency. As this is designed to be an online game the construction of a building can take several hours or days in real time.
# You will be given an initial ERD (Entity Relationship Diagram) as well as a .zip file containing test data stored as .csv (comma-separated format) files. From this information, and following the Tasks below, you will expand the database design, populate the database with test data, test the data, and answer questions in an online quiz for your final mark.

import psycopg2
import pandas as pd
# 获得连接
# 2.1 Task 1 - Setup the database
# 1. Download the test-data.zip from Canvas and extract the csv files in it.
# 2. Create a new file named assignment3.sql. In this file you will save all the sql code that you write.
# 3. Create a new postgres user named fsad. (Note: If you are using the lab machines, the username and database that has already been created for you is fine to use).
# 4. Create a new database named assignment3 <studentId> (where studentId is your email prefix, e.g. the abc123 part of abc123@student.bham.ac.uk).
# 5. Make sure the fsad user has full privileges on the assignment3 database.
conn = psycopg2.connect(database="postgres", user="postgres", password="123456", host="127.0.0.1", port="5433")
# 获得游标对象
cursor = conn.cursor()
# sql语句
# sql = "SELECT VERSION()"
sql = """SELECT * FROM player 
        ORDER BY player_name 
        ASC """
# 执行语句
cursor.execute(sql)
# 获取单条数据.
# data = cursor.fetchone()
# data = cursor.fetchmany(30)
data = cursor.fetchall()
# 打印
# print("database version : %s " % data)
print(data)
print("=========================")

[(6, 'AishaKhan', 'password6'), (4, 'FatimaAhmed', 'password4\r'), (1, 'HarrySmith', 'password1\r'), (5, 'MohammedAli', 'password5\r'), (3, 'OliverTaylor', 'password3\r'), (2, 'SophieBrown', 'password2\r')]


In [111]:
# Task 4 - Create Views
# Create a view for each building named <building info> that displays the following columns: 
# Name, Description, Resources Generated, Required Gold, Build Time.
sql2 = """CREATE OR REPLACE VIEW public.building_info
 AS
 SELECT building.building_name AS name,
    building.building_description AS description,
    resource.resource_name AS resources_generated,
    resource.resource_name='Gold' AS required_gold,
    buildingcost.build_time
   FROM building,
    resource,
    buildingcost
  WHERE building.building_id = buildingcost.building_id
  and building.resource_generated = resource.resource_id;"""
s = "select * from building_info"
cursor.execute(s)
# cursor.fetchall()
pd.DataFrame(cursor.fetchall(), columns=['name', 'description', 'resources_generated', 'required_gold', 'build_time'])
# print(data2)
# print("=========================")

,name,description,resources_generated,required_gold,build_time
0,Farm,Produces food for the kingdom,Food,False,3 days
1,Lumber Mill,Produces wood for the kingdom,Wood,False,2 days
2,Stone Quarry,Produces stone for the kingdom,Stone,False,5 days
3,Gold Mine,Produces gold for the kingdom,Gold,True,7 days
4,Marketplace,Allows for the exchange of resources between k...,Gold,True,14 days


In [112]:
# Create a view for each Player named <player name>buildings that displays all the buildings (by name) 
# the player has built in their base.
sql3 = """-- View: public.player_name

-- DROP VIEW public.player_name;

CREATE OR REPLACE VIEW public.player_name
 AS
 SELECT t.player_name,
    t3.building_name,
    t1.base_id,
    t2.building_id
   FROM player t
     LEFT JOIN base t1 ON t.player_id = t1.player_id
     LEFT JOIN basebuilding t2 ON t1.base_id = t2.base_id
     LEFT JOIN building t3 ON t2.building_id = t3.building_id
  GROUP BY t.player_name, t3.building_name, t1.base_id, t2.building_id;

ALTER TABLE public.player_name
    OWNER TO postgres;"""
s = "select * from player_name"
cursor.execute(s)
#cursor.fetchall()
pd.DataFrame(cursor.fetchall(), columns=['player_name', 'building_name', 'base_id', 'building_id'])
# 事物提交
# conn.commit()
# # 关闭数据库连接
# conn.close()

,player_name,building_name,base_id,building_id
0,AishaKhan,Stone Quarry,6,3
1,HarrySmith,Marketplace,1,5
2,MohammedAli,Farm,5,1
3,OliverTaylor,Gold Mine,3,4
4,FatimaAhmed,Lumber Mill,4,2
5,HarrySmith,Farm,1,1
6,AishaKhan,Lumber Mill,6,2
7,AishaKhan,Farm,6,1
8,SophieBrown,Stone Quarry,2,3
9,OliverTaylor,Lumber Mill,3,2


In [113]:
# # Create a view for each Player named <player >resources that displays the following columns: 
# # Name, Food Amount, Wood Amount, Stone Amount.
sql4 = """-- View: public.player_resources

-- DROP VIEW public.player_resources;

CREATE OR REPLACE VIEW public.player_resources
 AS
 SELECT t.player_name,
    sum(
        CASE
            WHEN t2.resource_name = 'Food'::text THEN t1.amount
            ELSE 0
        END) AS food_amount,
    sum(
        CASE
            WHEN t2.resource_name = 'Wood'::text THEN t1.amount
            ELSE 0
        END) AS wood_amount,
    sum(
        CASE
            WHEN t2.resource_name = 'Stone'::text THEN t1.amount
            ELSE 0
        END) AS stone_amount
   FROM player t
     LEFT JOIN playerresource t1 ON t.player_id = t1.player_id
     LEFT JOIN resource t2 ON t1.resource_id = t2.resource_id
  GROUP BY t.player_name;

ALTER TABLE public.play_resources
    OWNER TO postgres;"""
s = "select * from player_resources"
cursor.execute(s)
pd.DataFrame(cursor.fetchall(), columns=['player_name', 'food_amount', 'wood_amount', 'stone_amount'])

,player_name,food_amount,wood_amount,stone_amount
0,MohammedAli,5500,3500,800
1,AishaKhan,7000,4000,2000
2,OliverTaylor,6000,3000,1500
3,HarrySmith,5000,2500,1000
4,SophieBrown,4500,2000,500
5,FatimaAhmed,4000,1500,2000


In [114]:

#Create a view for each Player named <player >currencies that displays the following columns: Name, Gold Amount, Ethereal Silver Amount, Diamonds Amount.
sql5 = """-- View: public.player_currencies

-- DROP VIEW public.player_currencies;

CREATE OR REPLACE VIEW public.player_currencies
 AS
 SELECT t.player_name,
    sum(
        CASE
            WHEN t2.currency_name = 'Gold'::text THEN t1.amount
            ELSE 0
        END) AS gold_amount,
    sum(
        CASE
            WHEN t2.currency_name = 'Ethereal Silver'::text THEN t1.amount
            ELSE 0
        END) AS ethereal_silver_amount,
    sum(
        CASE
            WHEN t2.currency_name = 'Diamonds'::text THEN t1.amount
            ELSE 0
        END) AS diamonds_amount
   FROM player t
     LEFT JOIN playercurrency t1 ON t.player_id = t1.player_id
     LEFT JOIN currency t2 ON t1.currency_id = t2.currency_id
  GROUP BY t.player_name;

ALTER TABLE public.play_currencies
    OWNER TO postgres;"""
s = "select * from player_currencies"
cursor.execute(s)
pd.DataFrame(cursor.fetchall(), columns=['player_name', 'gold_amount', 'ethereal_silver_amount', 'diamonds_amount'])

,player_name,gold_amount,ethereal_silver_amount,diamonds_amount
0,MohammedAli,75,30,8
1,AishaKhan,300,150,50
2,OliverTaylor,50,20,5
3,HarrySmith,100,50,10
4,SophieBrown,200,100,20
5,FatimaAhmed,150,75,0


In [115]:
# Task 5 - Create Code Block
# Warning!
# Before you create and execute the code block make a backup of your database. Once the code block has been tested and is working RESTORE the database 
# from your backup.
# Create a code (DO) block that essentially fast-forwards any outstanding constructions by doing the following:
# 1. Declares a variable name test date.
# 2. Assigns this variable to the 10th of April 2023.
# 3. Checks each row in the construction queue table.
# 4. If the construction has NOT been completed (based on the start date plus build time being after the test date) 
# it should be removed from the construction queue table and inserted into the base building table.
# 5. Once the DO block is working correctly restore your database back to its state before running the DO block.
sql6 =""" -- 声明测试日期变量
DO $$
DECLARE 
    test_date DATE := '2023-04-10';
    construction RECORD;
BEGIN
    -- 循环遍历 constructionqueue 表中的每一行
    FOR construction IN SELECT * FROM constructionqueue
    LOOP
        -- 如果建筑尚未完成，则将其从 constructionqueue 表中删除并插入到 basebuilding 表中
        IF (construction.start_date + (SELECT build_time FROM buildingcost WHERE building_id = construction.building_id)) > test_date THEN
            INSERT INTO basebuilding (base_id, building_id) VALUES (construction.base_id, construction.building_id);
            DELETE FROM constructionqueue WHERE base_id = construction.base_id AND building_id = construction.building_id;
        END IF;
    END LOOP;
END $$;"""
cursor.execute(sql6)
# 事物提交



In [116]:
sqll = """select t.player_name, t3.building_name
from player t
left join base t1
on t.player_id = t1.player_id

left join basebuilding t2
on t1.base_id = t2.base_id

left join building t3
on t2.building_id = t3.building_id

where t.player_name = 'SophieBrown' 
group by t.player_name, t3.building_name;"""
cursor.execute(sqll)
cursor.fetchall()

[('SophieBrown', 'Farm'),
 ('SophieBrown', 'Gold Mine'),
 ('SophieBrown', 'Lumber Mill'),
 ('SophieBrown', 'Marketplace'),
 ('SophieBrown', 'Stone Quarry')]

- 以下为测试

In [117]:
sql6 = "SELECT * FROM player ORDER BY player_name ASC LIMIT 1;"
cursor.execute(sql6)
cursor.fetchall()

[(6, 'AishaKhan', 'password6')]

In [118]:
cursor.execute("SELECT * FROM constructionqueue;")
cursor.fetchall()
#pd.DataFrame(cursor.fetchall(), columns=['base_id', 'building_id', 'start_date'])

[]

In [119]:
cursor.execute("SELECT building_id, build_time FROM buildingcost ORDER BY build_time DESC LIMIT 4;")
cursor.fetchall()

[(5, datetime.timedelta(days=14)),
 (4, datetime.timedelta(days=7)),
 (3, datetime.timedelta(days=5)),
 (1, datetime.timedelta(days=3))]

In [120]:
cursor.execute("SELECT * from currency WHERE premium = true;")
cursor.fetchall()

[(3, 'Diamonds', True)]

In [121]:
cursor.execute("SELECT * from basebuilding")
# cursor.fetchall()
pd.DataFrame(cursor.fetchall(), columns=['base_id', 'building_id'])

,base_id,building_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,1,2
6,2,2
7,3,2
8,1,3
9,2,3


In [122]:
# 事物提交
conn.commit() 
# 关闭数据库连接
conn.close()